# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 

## Final Project Group 40: Trump Tweets and Market Volatility


**Harvard University**<br/>
**Fall 2019**<br/>
**Group**: Dylan DeMarco, Daiki Ina, Roger Zhang, Jerry Huang

<hr style="height:2pt">

In [2]:
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

import pandas_profiling

# Goal 1: Gather trump tweets from twitter and save to a csv

## Setup a twitter app to get through the API

In [4]:
# Skip and use repo from project document

## Pull all of the tweets

In [5]:
"""
Sourced from: http://www.trumptwitterarchive.com/archive
Exported manually with all features/info to csv with the query parameter: date > 11/09/2016 -- day 1 after election  
"""
collection_date = '2019-11-07'
tweet_df = pd.read_csv('./data/tweets_scraped_11_07_2019.csv')
tweet_df

tweet_df.to_csv('./data/raw/tweet_data.csv')
pandas_profiling.ProfileReport(tweet_df)

## Filter to find DJT's *true* tweets

In [6]:
# REALLY good resource: http://varianceexplained.org/r/trump-tweets/



## Gather VIX data for the same window of time

In [7]:
"""
Sourced from the creators of vix (trustworthy source IMO) -- cboe
http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/vix-historical-data
download 2004 -> present and subset the csv below.
"""
vix_full = pd.read_csv('./data/vix_scraped_11_08_2019.csv')
vix_full = vix_full.rename(columns=vix_full.iloc[0]).drop(vix_full.index[0])
vix_full.Date = pd.to_datetime(vix_full.Date)

vix_election_to_present = vix_full[vix_full['Date'] >= pd.Timestamp('11/09/2016')]
vix_election_to_present.to_csv('./data/raw/vix_data.csv')
vix_election_to_present

,Date,VIX Open,VIX High,VIX Low,VIX Close
3238,2016-11-09,20.70,21.48,14.33,14.38
3239,2016-11-10,14.01,16.30,13.26,14.74
3240,2016-11-11,14.83,16.00,14.15,14.17
3241,2016-11-14,14.69,15.56,14.39,14.48
3242,2016-11-15,14.16,14.65,13.30,13.37
...,...,...,...,...,...
3987,2019-11-01,12.99,13.12,12.25,12.30
3988,2019-11-04,12.68,13.13,12.44,12.83
3989,2019-11-05,12.85,13.28,12.25,13.10
3990,2019-11-06,13.18,13.39,12.60,12.62


## Other sources?? Headline tracking?